# Домашнее задание №2 по plotly
**Исполнитель:**  Behzod Jumaev


In [1]:
import numpy as np
import pandas as pd
from connector import connect_to
import plotly.express as px

## Задание №1  
1. Создайте датафрейм содержащий сумму и количество продаж треков в разбивке по странам и континентам.  
2. Создайте scatter plot показывающий соотношение суммы продаж и количества треков.  
3. Добавьте разделение по цвету на основе континента. Цвет Азии должен быть **жёлтым**, цвет Европы **синим**, цвет Северной Америки **красным**.
4. При наведении на точки данных в диаграмме должна всплывать информация о стране продажи (жирным), сумме продаж, количестве треков, средней сумме на один трек
5. Добавьте название диаграмме. Шрифт должен быть 30, цвет шрифта серый.
6. Измените шрифт тиков (значения осей) на 14, цвет на серый.

In [11]:
country_continent = {
    'Germany': 'Европа',
    'Norway': 'Европа',
    'Belgium': 'Европа',
    'Canada': 'Северная Америка',
    'USA': 'Северная Америка',
    'France': 'Европа',
    'Ireland': 'Европа',
    'United Kingdom': 'Европа',
    'Australia': 'Австралия',
    'Chile': 'Южная Америка',
    'India': 'Азия',
    'Brazil': 'Южная Америка',
    'Portugal': 'Европа',
    'Netherlands': 'Европа',
    'Spain': 'Европа',
    'Sweden': 'Европа',
    'Czech Republic': 'Европа',
    'Finland': 'Европа',
    'Denmark': 'Европа',
    'Italy': 'Европа',
    'Poland': 'Европа',
    'Austria': 'Европа',
    'Hungary': 'Европа',
    'Argentina': 'Южная Америка'
}


In [12]:
query = '''
    SELECT 
        i.billing_country, 
        SUM(il.unit_price * il.quantity) AS sales_sum,
        COUNT(il.track_id) AS track_count
    FROM invoice i
    JOIN invoice_line il ON i.invoice_id = il.invoice_id
    GROUP BY i.billing_country
'''


In [13]:
with connect_to('postgres') as pg:
    sales_summary = pd.read_sql(query, pg)
    

In [14]:
sales_summary.head()

,billing_country,sales_sum,track_count
0,Argentina,37.62,38
1,Spain,37.62,38
2,Italy,37.62,38
3,Hungary,45.62,38
4,India,75.26,74


In [15]:
sales_summary['continent'] = sales_summary['billing_country'].map(country_continent)

In [24]:
fig1 = px.scatter(
    sales_summary,
    x='track_count',
    y='sales_sum',
    color='continent',
    color_discrete_map={
        'Asia': 'yellow',
        'Europe': 'blue',
        'North America': 'red'
    },
    hover_data={
        'billing_country': True,
        'sales_sum': True,
        'track_count': True,
        'avg_sales_per_track': round(sales_summary['sales_sum'] / sales_summary['track_count'], 3)
    },
    title="Сравнение количества продаж и суммы по странам и континентам"
)


In [25]:
fig1.update_traces(
    marker=dict(size=12))

fig1.update_layout(
    title={
        'text': "Сравнение количества продаж и суммы по странам и континентам",
        'font': {'size': 30, 'color': "grey"}
    },
    xaxis={
        'title': 'Количество треков',
        'tickfont': {'size': 14, 'color': "grey"}
    },
    yaxis={
        'title': 'Сумма продаж',
        'tickfont': {'size': 14, 'color': "grey"}
    }
)


fig1.show()


## Задание №2  
1. Создайте датафрейм содержащий количество треков в разбивке по названиям жанров.
2. Создайте столбчатую диаграмму. По оси X должны быть названия жанров, по оси Y - количество треков.
3. Кастомизируйте диаграмму так, чтобы столбики всех жанров были серого цвета. За исключением жанров *Jazz*, *Rock* и *Alternative & Punk*. Эти жанры должны выделяться цветом. Цвета можете выбирать сами.
4. Уберите легенду.  
5. Добавьте название диаграмме. Название должно быть курсивным.
6. Фон диаграммы должен быть белым.  

In [125]:
query = '''
    SELECT 
        g.name AS genre_name,
        COUNT(t.track_id) AS track_count
    FROM track t
    JOIN genre g ON t.genre_id = g.genre_id
    GROUP BY g.name
'''


In [126]:
with connect_to('postgres') as pg:
    genre_summary = pd.read_sql(query, pg)

In [127]:
genre_summary.head()

,genre_name,track_count
0,Heavy Metal,28
1,TV Shows,93
2,Latin,579
3,Electronica/Dance,30
4,R&B/Soul,61


In [128]:
fig2 = px.bar(
    genre_summary,
    x='genre_name',
    y='track_count'
)

In [129]:
custom_colors = {
    'Jazz': 'orange',
    'Rock': 'purple',
    'Alternative & Punk': 'green'
}

In [130]:
fig2.update_traces(
    marker_color=[
        custom_colors[genre] if genre in custom_colors else 'gray'
        for genre in genre_summary['genre_name']
    ],
    showlegend=False
)

fig2.update_layout(
    title={
        'text': "Распределение треков по жанрам",
        'font': {'size': 24, 'color': "black", 'family': "Courier New, monospace"},
        'xref': "paper",
        'x': 0.5
    },
    xaxis={
        'title': 'Жанры',
        'tickfont': {'size': 14, 'color': "grey"}
    },
    yaxis={
        'title': 'Количество треков',
        'tickfont': {'size': 14, 'color': "grey"}
    },
    plot_bgcolor='white'
)

fig2.show()
